model 





In [3]:
import keras
#from cnnmodel import model
from keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import os, random, cv2
from keras.utils import to_categorical

ImportError: cannot import name 'normalize_data_format'

In [2]:
def createFileList(myDir):
    format = ('.tiff','.jpg','.png','.PNG')
    fileList = []
    print(os.walk(myDir, topdown=False))
    for root, dirs, files in os.walk(myDir, topdown=False):
        
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList 

In [3]:
classes={0:'angry', 1:'fear', 2:'happy', 3:'neutral', 4:'sad', 5:'surprise'}




batch_size = 32
epochs = 60

train_x = [] #(10000,1,227,227) (1,227,227)에 대해 append
train_y = [] #(10000, 6) 


#train_dir에 적용
#e.g. case sad
#path 읽어 = [(1,227,227)만들기 |  파싱하여 label 얻기] 

#train_x.append(q)
'''
z = load_img(path = './data/train/sad/KA.SA2.34.tiff', grayscale=True, target_size=(227,227),
    interpolation='nearest')
q=np.asarray(z).astype('float32')
print(np.asarray([q]).shape)
q = np.asarray([q])
'''

fileList = []
fileList = createFileList('./data/train')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    train_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    train_y.append(to_categorical(t, 6))

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
print(train_x.shape)   
print(train_y.shape)   

train_len = train_x.shape[0]

<generator object walk at 0x7fdb0f87ff10>
(1636, 1, 227, 227)
(1636, 6)


In [4]:
#이상  train_x/ train_y 구성 끝

#val_x/ val_y 도 일단 똑같이 구성

val_x = []
val_y = []


fileList = createFileList('./data/valid')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    val_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    val_y.append(to_categorical(t, 6))

val_x = np.asarray(val_x)
val_y = np.asarray(val_y)
print(val_x.shape)   
print(val_y.shape)   

val_len = train_x.shape[0]


<generator object walk at 0x7fdb0f87ffc0>
(199, 1, 227, 227)
(199, 6)


In [5]:
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=10,
    shear_range=0.1,
    fill_mode = 'constant',
    cval = 0,
    horizontal_flip=True)

print(train_x.shape)
train_x_moveaxis = np.moveaxis(train_x, 1, 3)  
print(train_x_moveaxis.shape)
train_datagen.fit(train_x_moveaxis)

train_x_mean = train_datagen.mean

(1636, 1, 227, 227)
(1636, 227, 227, 1)


In [6]:
print(val_x.shape)

(199, 1, 227, 227)


In [7]:
val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255, 
)
val_x_moveaxis = np.moveaxis(val_x, 1, 3)  
val_datagen.fit(val_x_moveaxis)
#val_datagen.mean = x꺼 배끼끼
print(val_datagen.mean)
val_datagen.mean = train_x_mean
print(val_datagen.mean)

[[[118.370995]]]
[[[119.09373]]]


In [8]:
test_x = []
test_y = []


fileList = createFileList('./data/test')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    test_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    test_y.append(to_categorical(t, 6))

test_x = np.asarray(test_x)
test_y = np.asarray(test_y)
print(test_x.shape)   
print(test_y.shape)   

test_len = train_x.shape[0]


test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255, 
)
test_x_moveaxis = np.moveaxis(test_x, 1, 3)  
test_datagen.fit(test_x_moveaxis)
#val_datagen.mean = x꺼 배끼끼
print(test_datagen.mean)
test_datagen.mean = train_x_mean
print(test_datagen.mean)

<generator object walk at 0x7fdb0f8d1048>
(223, 1, 227, 227)
(223, 6)
[[[115.71098]]]
[[[119.09373]]]


In [9]:
#sequential model definition
#model.compile
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta, Adam
from keras.layers.normalization import BatchNormalization


def model_generate():
    img_rows, img_cols = 227, 227
    model = Sequential()
    model.add(Convolution2D(36, 11, 11, border_mode='valid',
                            input_shape=(img_rows, img_cols,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(2, 2), dim_ordering='tf'))
    model.add(MaxPooling2D(pool_size=(5, 5),strides=(2, 2)))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf')) 
    
    model.add(Convolution2D(96, 5, 5))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf')) 
    
    model.add(Convolution2D(144, 3, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=(2, 2)))
     
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf'))
    model.add(Convolution2D(96, 3, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf'))
    model.add(Convolution2D(128, 3, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
     
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf'))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=(2, 2)))
     
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
     
      
    model.add(Dense(6))
    model.add(BatchNormalization())
      
    model.add(Activation('softmax'))

    ada = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    #model = multi_gpu_model(model, gpus=2)
    model.compile(loss='categorical_crossentropy',
                  optimizer=ada,
                  metrics=['accuracy'])
    model.summary()
    return model

model = model_generate()


filepath='model7_total_#4set_kwang3_2_7_2.hdf5'
checkpointer = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')



/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (11, 11), input_shape=(227, 227,..., padding="valid")`
  
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(2, 2), data_format="channels_last")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_last")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (5, 5))`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_last")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 217, 217, 36)      4392      
_________________________________________________________________
batch_normalization_1 (Batch (None, 217, 217, 36)      144       
_________________________________________________________________
activation_1 (Activation)    (None, 217, 217, 36)      0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 221, 221, 36)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 109, 109, 36)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 111, 111, 36)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 107, 107, 96)      86496     
__________

In [11]:
model.fit_generator(train_datagen.flow(train_x_moveaxis, train_y, batch_size =batch_size),
                    #steps_per_epoch = train_len/batch_size*12,
                    steps_per_epoch = train_len/batch_size*4,
                    epochs = epochs,
                    validation_data = val_datagen.flow(val_x_moveaxis,val_y,batch_size = batch_size),
                    callbacks = [checkpointer])

#l = train_datagen.flow(train_x_moveaxis, train_y, batch_size =batch_size)

#next(l)[0].shape

Epoch 1/60


FailedPreconditionError: Attempting to use uninitialized value Adam/iterations
	 [[Node: Adam/iterations/read = Identity[T=DT_INT64, _class=["loc:@Adam/iterations"], _device="/job:localhost/replica:0/task:0/cpu:0"](Adam/iterations)]]

Caused by op 'Adam/iterations/read', defined at:
  File "/opt/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  Fi

In [ ]:
#classes={0:'angry', 1:'fear', 2:'happy', 3:'neutral', 4:'sad', 5:'surprise'}
#vd = val_datagen.flow(val_x_moveaxis,val_y,batch_size = 1)
#sample = vd.next()
#sample_x,sample_y = sample
#print(sam)

"""
test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255, 
)
#print("0hi")
#print(test_datagen.mean)
test_datagen.fit(val_x_moveaxis[:10])
#print(test_datagen.mean)
test_datagen.mean = val_datagen.mean
#print(test_datagen.mean)
"""


e = model.predict_generator(test_datagen.flow(test_x_moveaxis,test_y,batch_size = 1,shuffle= False))
print(val_datagen.mean)
print([np.argmax(pre) for pre in e])
#print(val_x_moveaxis[0])
#print(val_y)
#print(e)
print([np.argmax(x) for x in test_y])

In [ ]:
correct = 0
for x,y in zip(e,test_y):
    if np.argmax(x) == np.argmax(y):
        correct+=1
    else: continue
        
print(float(correct)/len(test_y)*100)

A total data # * steps for epochs / batchsize  ==  172 
Therefore, 172 * 32(batchsize) == A total data #

In [ ]:

valid_x = [] #(10000,1,227,227) (1,227,227)에 대해 append
valid_y = [] #(10000, 6) 


fileList = []
fileList = createFileList('./data/valid')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    valid_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    valid_y.append(to_categorical(t, 6))

    
valid_x = np.asarray(valid_x)
valid_y = np.asarray(valid_y)
print(valid_x.shape)   
print(valid_y.shape)   
valid_x_moveaxis = np.moveaxis(valid_x, 1, 3)  
valid_x_moveaxis.shape


In [ ]:

valid_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255, 
)
#print("0hi")
#print(test_datagen.mean)
valid_datagen.fit(valid_x_moveaxis[:])
print(valid_datagen.mean)
valid_datagen.mean = [[[0]]]
print(valid_datagen.mean)


TEST CODE
e1 = model.predict_generator(valid_datagen.flow(valid_x_moveaxis,valid_y,batch_size = 1,shuffle= False))
print(valid_datagen.mean)
print([np.argmax(pre) for pre in e1])
#print(val_x_moveaxis[0])
#print(val_y)
print(e1)
print([np.argmax(x) for x in valid_y])


correct = 0
for x,y in zip(e1,valid_y):
    if np.argmax(x) == np.argmax(y):
        correct+=1
    else: continue
        
print(float(correct)/len(valid_y)*100)


